# Acidentes de tráfico de bicicletas

Vamos a utilizar un dataset de accidentes de tráfico de bicicletas que provienen del portal de datos abiertos de Madrid

https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=20f4a87ebb65b510VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default


Una vez preprocesados los datos tendremos un dataset con los siguientes campos:


- Número de expediente del accidente
- Fecha del accidente
- Distrito donde ha ocurrido el accidente
- Tipo de accidente
- Sexo de la persona que ha tenido el accidente
    

Vamos a intentar resolver las siguientes preguntas:

- Datos del número de expediente 2020S000044
- Accidentes que ocurren en el distrito de Moratalaz ordenados por fecha (Descendente)
- Número de accidentes por tipo de accidente (sin ningún orden establecido)
- Número de accidentes cuyo tipo es Colisión frontal

Preprocesado de datos

Se realizará un preprocesado de datos cuyo objetico es limpiar, simplificar y normalizar los datos para que la prática resulte más sencilla

El formato de salida será un fichero Microsoft Excel que contendrá los datos limpios.



## Carga de datos

In [1]:
import pandas as pd

Leemos los datos ...

In [2]:
df = pd.read_csv("data/AccidentesBicicletas_2020.txt", delimiter=";")

## Exploración y limpieza

Limpiamos los datos ...

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 775 entries, 0 to 774
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NumExpediente        775 non-null    object 
 1   Fecha                775 non-null    object 
 2   Hora                 775 non-null    object 
 3   Calle                775 non-null    object 
 4   Numero               775 non-null    object 
 5   Distrito             775 non-null    object 
 6   TipoAccidente        775 non-null    object 
 7   EstadoMetereologico  759 non-null    object 
 8   TipoVehiculo         775 non-null    object 
 9   TipoPersona          775 non-null    object 
 10  RangoEdad            775 non-null    object 
 11  Sexo                 763 non-null    object 
 12  Lesividad            684 non-null    float64
dtypes: float64(1), object(12)
memory usage: 78.8+ KB


In [4]:
df = df.assign(Fecha = pd.to_datetime(df.Fecha, format="%d/%m/%Y"),
               Min = pd.to_numeric(df.Hora.str.split(":").str.get(1)),
               Hora = pd.to_numeric(df.Hora.str.split(":").str.get(0))
              )
df = df.filter(["NumExpediente", "Fecha", "Distrito", "TipoAccidente", "Sexo"])
# Ponemos las columnas en minúsculas para no se inserten en Postgres con Dobles Comillas
df = df.rename(columns = {"NumExpediente" : "num_expediente",
                           "Fecha" : "fecha",
                           "Distrito": "distrito",
                           "TipoAccidente" : "tipo_accidente",
                           "Sexo": "sexo"}
              )
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 775 entries, 0 to 774
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   num_expediente  775 non-null    object        
 1   fecha           775 non-null    datetime64[ns]
 2   distrito        775 non-null    object        
 3   tipo_accidente  775 non-null    object        
 4   sexo            763 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 30.4+ KB


In [5]:
df.sample(10)

,num_expediente,fecha,distrito,tipo_accidente,sexo
454,2020S009961,2020-07-16,ARGANZUELA,Caída,Hombre
513,2020S010760,2020-08-01,VILLA DE VALLECAS,Choque contra obstáculo fijo,Hombre
324,2020S008367,2020-06-16,HORTALEZA,Caída,Hombre
49,2020S002409,2020-01-31,VILLAVERDE,Caída,Hombre
736,2020S014666,2020-10-18,ARGANZUELA,Caída,Mujer
740,2020S014766,2020-10-19,MONCLOA-ARAVACA,Colisión fronto-lateral,Hombre
689,2020S013804,2020-10-03,MONCLOA-ARAVACA,Caída,Mujer
430,2020S009930,2020-07-08,LATINA,Colisión lateral,Hombre
279,2020S007862,2020-06-05,MONCLOA-ARAVACA,Caída,Hombre
294,2020S008055,2020-06-09,USERA,Colisión frontal,Hombre


Eliminamos los expedientes duplicados para simplificar el ejemplo ...

In [6]:
ExpedientesDuplicados = pd.DataFrame(df.groupby("num_expediente").size()).reset_index().rename( columns = {0 : 'n'}).query("n > 1").num_expediente
df = df[~df.num_expediente.isin(ExpedientesDuplicados)]
df = df[df.sexo.notnull()]
df = df.reset_index(drop = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   num_expediente  614 non-null    object        
 1   fecha           614 non-null    datetime64[ns]
 2   distrito        614 non-null    object        
 3   tipo_accidente  614 non-null    object        
 4   sexo            614 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 24.1+ KB


In [7]:
df.sample(10)

,num_expediente,fecha,distrito,tipo_accidente,sexo
269,2020S008630,2020-06-19,USERA,Colisión fronto-lateral,Hombre
151,2020S006942,2020-05-17,MONCLOA-ARAVACA,Caída,Hombre
96,2020S005296,2020-03-07,CENTRO,Choque contra obstáculo fijo,Hombre
214,2020S007762,2020-06-03,LATINA,Caída,Hombre
376,2020S010103,2020-07-20,CIUDAD LINEAL,Colisión fronto-lateral,Hombre
340,2020S009498,2020-07-07,RETIRO,Alcance,Mujer
391,2020S010363,2020-07-24,TETUÁN,Colisión frontal,Hombre
565,2020S014302,2020-10-11,LATINA,Caída,Hombre
387,2020S010311,2020-07-23,VILLAVERDE,Colisión fronto-lateral,Hombre
535,2020S013667,2020-10-01,CHAMARTÍN,Colisión lateral,Mujer


## Grabación de los datos limpios

Y finalmente escribimos el resultado a un fichero Excel ,,,

In [8]:
writer = pd.ExcelWriter("data/accidentes.xlsx")
df.to_excel(writer, 'Data', index=False)
writer.save()